The subject of this EDA workbook is a raw 1,000,000 row x 276 column data set. The approach for exploring the data is domain agnostic; so little to no prior knowledge of the data set and its structure is assumed. There are three main steps: 
1. An initial data scrub eliminates columns of no or little value. The question being answered in this step is not which columns are important, but rather which columns are certainly not important or redundant. 
2. Creating a summary table of the edited raw data set. The summary table cites the number of non-nulls, the number of unique values, and the datatype in each column.
3. Filtering or grouping column headers by the three columns in the summary table will facilitate inspecting smaller tables and maybe highlight similarities or additional avenues to explore. 
The final part of this workbook plots a cumulative distribution function of the unique value counts (ordered from largest to smallest) of one or more columns. The cdf may be a useful tool in understanding how the data in a column of categorical value is distributed, allowing bucketing in a more informed way.

In [1]:
%matplotlib notebook
import psycopg2
import pandas as pd
import seaborn as sns
import warnings
import matplotlib as plot
import matplotlib.pyplot as plt

warnings.simplefilter('ignore')
pd.options.display.max_columns = 300
sns.set_style('darkgrid')

In [84]:
conn = psycopg2.connect(database='usaspending', user='team', password='ZAQ!@#zaq123', host='dopelytics.site', port='5432')

In [87]:
sql_cols = ('federal_action_obligation, '
            #'total_dollars_obligated, '
            'base_and_exercised_options_value, '
            'base_and_all_options_value, '
            #'awarding_sub_agency_name, '
            'awarding_sub_agency_code, '
            #'awarding_office_name, '
            'awarding_office_code, '
            #'funding_sub_agency_name, '
            'funding_sub_agency_code, '
            #'funding_office_name, '  too many NaN
            'primary_place_of_performance_state_code, '
            'award_or_idv_flag, '
            #'award_type, '
            'award_type_code, '
            #'type_of_contract_pricing, '
            'type_of_contract_pricing_code, '
            #'dod_claimant_program_description, '
            'dod_claimant_program_code, '
            'type_of_set_aside_code, '
            #'multi_year_contract, ' too many NaN
            #'dod_acquisition_program_description, ' too many NaN
            #'subcontracting_plan, ' too many NaN
            #'contract_bundling, '
            'contract_bundling_code, '
            #'evaluated_preference, ' too many NaN
            #'national_interest_action, '
            'national_interest_action_code, '
            #'cost_or_pricing_data, ' too many NaN
            #'gfe_gfp, '
            'gfe_gfp_code, '
            #'contract_financing, '
            'contract_financing_code, '
            'portfolio_group, '
            #'product_or_service_code_description, '
            'product_or_service_code, '
            #'naics_bucket_title, ' too many NaN
            #'naics_description'
            'naics_code'
            )

In [88]:
# Create dataframe
sql_tbl_name = 'consolidated_data2'
df = pd.read_sql_query('SELECT ' + sql_cols + ' FROM ' + sql_tbl_name, con=conn)
print('Shape of initial df:', df.shape)

Shape of initial df: (272790, 19)


In [89]:
# Drop rows with NaN values
df = df.dropna()
print('Shape with no NaN values:', df.shape)

Shape with no NaN values: (270917, 19)


In [4]:
million_row_sample_path = '../SpendingData/2015_097_Contracts_Full_20191009/2015_097_Contracts_Full_20191010_1.csv'

In [5]:
raw_data = pd.read_csv(million_row_sample_path) #million_row_sample_path is where the file downloaded from USASpending web app should be saved.

In [122]:
raw_data['type_of_set_aside'].value_counts()

NO SET ASIDE USED.                                         870135
SMALL BUSINESS SET ASIDE - TOTAL                            96636
8(A) SOLE SOURCE                                             8454
8A COMPETED                                                  6512
SMALL BUSINESS SET ASIDE - PARTIAL                           5501
SERVICE DISABLED VETERAN OWNED SMALL BUSINESS SET-ASIDE      3448
8(A) SOLE SOURCE                                             2961
HUBZONE SET-ASIDE                                            1867
WOMEN OWNED SMALL BUSINESS                                    460
SDVOSB SOLE SOURCE                                            144
ECONOMICALLY DISADVANTAGED WOMEN OWNED SMALL BUSINESS         114
HUBZONE SOLE SOURCE                                            41
HUBZONE SOLE SOURCE                                             6
8(A) WITH HUB ZONE PREFERENCE                                   3
HBCU OR MI SET-ASIDE -- PARTIAL                                 3
HBCU OR MI

In [123]:
raw_data['type_of_set_aside_code'].value_counts()

NONE       870135
SBA         96636
8AN         11415
8A           6512
SBP          5501
SDVOSBC      3448
HZC          1867
WOSB          460
SDVOSBS       144
EDWOSB        114
HZS            47
HS3             3
HMP             3
HMT             2
Name: type_of_set_aside_code, dtype: int64

In [6]:
raw_data['psc_description'] = raw_data['product_or_service_code_description'].copy() #renaming so it doesn't get deleted for having 'code' in header

In [41]:
def initial_scrub(input_df):
    '''description'''
    raw_data_df = input_df.copy()
    list_of_code_headers = list(raw_data_df.filter(regex='code'))
    no_code_df = raw_data_df[raw_data_df.columns.drop(list_of_code_headers)].copy()
    list_of_recipient_headers = list(no_code_df.filter(regex='recipient'))
    no_recipient_df = no_code_df[no_code_df.columns.drop(list_of_recipient_headers)].copy()
    list_of_business_headers = list(no_recipient_df.filter(regex='business'))
    no_business_df = no_recipient_df[no_recipient_df.columns.drop(list_of_business_headers)].copy()
    #need to drop other columns assciated with award winner, since these aren't known when award is announced
    return no_business_df, list_of_code_headers, list_of_recipient_headers, list_of_business_headers

In [42]:
def data_summary(input_df):
    '''this function produces a summary of a raw data set. the summary contains three facts about each column of the
    raw dataset: 1. the non-null count 2. the unique values contained in the column 3. the data type contained in the
    column; this data type is determined by pandas, I think, and might not be exactly correct all of the time'''
    working_df = input_df.copy()
    column_null_counts = working_df.isnull().sum()
    column_non_null_counts = len(working_df)-column_null_counts
    unique_entries_per_column = working_df.nunique()
    data_type = working_df.dtypes
    summary_df = pd.concat([column_non_null_counts,unique_entries_per_column,data_type],axis=1)
    summary_df.columns =  ['non_null_counts','unique_value_counts','data_type']
    return summary_df

In [43]:
smaller_raw_df, dropped_code_headers, dropped_recipient_headers, dropped_business_headers = initial_scrub(raw_data)

In [44]:
len(dropped_code_headers)

63

In [45]:
len(dropped_recipient_headers)

13

In [46]:
len(dropped_business_headers)

22

In [47]:
smaller_raw_summary = data_summary(smaller_raw_df)

In [48]:
smaller_raw_summary.head()

,non_null_counts,unique_value_counts,data_type
contract_transaction_unique_key,1000000,1000000,object
award_id_piid,1000000,376481,object
modification_number,1000000,2330,object
transaction_number,1000000,19,int64
parent_award_agency_id,780145,30,object


In [49]:
smaller_raw_summary.data_type.value_counts()

object     154
float64     22
int64        3
Name: data_type, dtype: int64

In [50]:
object_column_df = smaller_raw_summary[smaller_raw_summary.data_type=='object']
float64_column_df = smaller_raw_summary[smaller_raw_summary.data_type=='float64']
int64_column_df = smaller_raw_summary[smaller_raw_summary.data_type=='int64']

In [51]:
int64_column_df

,non_null_counts,unique_value_counts,data_type
transaction_number,1000000,19,int64
action_date_fiscal_year,1000000,1,int64
number_of_actions,1000000,238,int64


In [52]:
float64_column_df.shape

(22, 3)

Many of the columns that contain data type float64 (above table) appear to be monetary

In [53]:
raw_data.shape

(1000000, 277)

In [54]:
float64_column_df_null_filtered = float64_column_df[float64_column_df.non_null_counts>0.2*raw_data.shape[0]]

In [55]:
float64_column_df_null_filtered.shape

(10, 3)

In [56]:
object_column_df

,non_null_counts,unique_value_counts,data_type
contract_transaction_unique_key,1000000,1000000,object
award_id_piid,1000000,376481,object
modification_number,1000000,2330,object
parent_award_agency_id,780145,30,object
parent_award_agency_name,371252,25,object
...,...,...,...
highly_compensated_officer_3_name,26055,256,object
highly_compensated_officer_4_name,26041,251,object
highly_compensated_officer_5_name,26015,240,object
last_modified_date,1000000,69509,object


In [57]:
object_column_df_null_filtered = object_column_df[object_column_df.non_null_counts>0.2*raw_data.shape[0]]

In [58]:
object_column_df.shape

(154, 3)

In [60]:
object_column_df_null_filtered.shape

(141, 3)

In [36]:
object_column_df_null_filtered_binary = object_column_df_null_filtered[object_column_df_null_filtered.unique_value_counts==2]

In [61]:
object_column_df_null_filtered_no_binary = object_column_df_null_filtered[object_column_df_null_filtered.unique_value_counts!=2]

In [81]:
object_column_df_null_filtered_no_binary.unique_value_counts.describe()

count         68.000000
mean       39219.411765
std       170218.426224
min            1.000000
25%            4.000000
50%           14.000000
75%          710.500000
max      1000000.000000
Name: unique_value_counts, dtype: float64

In [75]:
box_plot_data = object_column_df_null_filtered_no_binary[object_column_df_null_filtered_no_binary.unique_value_counts<100].copy()

In [77]:
box_plot_data.shape

(46, 3)

In [ ]:
sns.boxplot(box_plot_data.unique_value_counts,orient='v')

In [37]:
object_column_df_null_filtered_binary

,non_null_counts,unique_value_counts,data_type
commercial_item_test_program,970999,2,object
a76_fair_act_action,384849,2,object
local_area_set_aside,384484,2,object
clinger_cohen_act_planning,384857,2,object
parent_award_single_or_multiple,352589,2,object
...,...,...,...
school_of_forestry,1000000,2,object
veterinary_college,1000000,2,object
dot_certified_disadvantage,1000000,2,object
c8a_program_participant,1000000,2,object


In [25]:
def get_filtered_summary_list(summary_df,max_null,min_unique,max_unique):
    null_edited_df = summary_df[summary_df.non_null_counts>=max_null].copy()
    unique_edited_df = null_edited_df[(null_edited_df.unique_value_counts>min_unique)&(null_edited_df.unique_value_counts<max_unique)].copy()
    #data_type_edited_df = unique_edited_df[unique_edited_df.data_type.isin(data_type)].copy()
    #data_type_edited_df = unique_edited_df[unique_edited_df.data_type==data_type].copy()
    column_list = unique_edited_df.index.tolist()
    print('the list is length '+str(len(column_list)))
    return unique_edited_df, column_list

In [26]:
edited_summary, test_list = get_filtered_summary_list(smaller_raw_summary,800000,3,50)

the list is length 19


In [27]:
edited_summary

,non_null_counts,unique_value_counts,data_type
transaction_number,1000000,19,int64
awarding_sub_agency_name,1000000,24,object
funding_agency_name,1000000,16,object
award_type,1000000,8,object
type_of_contract_pricing,1000000,16,object
contract_bundling,999478,4,object
dod_claimant_program_description,999994,27,object
recovered_materials_sustainability,998416,15,object
domestic_or_foreign_entity,916154,5,object
information_technology_commercial_item_category,997789,7,object


In [112]:
def column_count_cdf(data_df, column):
    column_summary = pd.DataFrame(columns=['count','percent','percent_running_sum'])
    column_summary['count'] = data_df[column].value_counts()
    nonnulls = data_df[column].value_counts().sum()
    column_summary['percent'] = data_df[column].value_counts()/nonnulls*100
    column_summary['percent_running_sum'] = column_summary['percent'].cumsum()
    #column_summary = column_summary.reset_index()
    #column_summary = column_summary.drop('index',axis=1)
    return column_summary

def plot_cdf(data_df, columnlist):
    max_x = 0
    for column in columnlist:
        print(column)
        colsummary=column_count_cdf(data_df, column)
        colsummary['percent_running_sum'].plot(linewidth=3)
        if max_x<=max(colsummary['count']):
            max_x = max(colsummary['count'])
        print(max_x)
    plt.legend(columnlist, fontsize=12)
    plt.ylim([-5,105])
    plt.xlim(xmin=-5)
    plt.title('Target Distribution CDF',fontsize=20)
    plt.xlabel('Target\'s Unique Value',fontsize=15)
    plt.ylabel('% of Values',fontsize=15)

In [115]:
set_aside_summary = column_count_cdf(df,'type_of_set_aside_code')

In [116]:
set_aside_summary_no_none = set_aside_summary

,count,percent,percent_running_sum
NONE,147731,54.529985,54.529985
SBA,76884,28.379171,82.909157
8AN,19629,7.245393,90.154549
8A,15205,5.612420,95.766969
SDVOSBC,4153,1.532942,97.299911
HZC,3833,1.414824,98.714736
WOSB,1998,0.737495,99.452231
SBP,814,0.300461,99.752692
EDWOSB,346,0.127714,99.880406
SDVOSBS,177,0.065334,99.945740


In [120]:
df[['type_of_set_aside_code','type_of_set_aside_description']]

KeyError: "['type_of_set_aside_description'] not in index"

In [ ]:
plot_cdf(smaller_raw_df,['naics_description'])

In [93]:
set_aside_summary

,count,percent,percent_running_sum
0,147731,54.529985,54.529985
1,76884,28.379171,82.909157
2,19629,7.245393,90.154549
3,15205,5.612420,95.766969
4,4153,1.532942,97.299911
5,3833,1.414824,98.714736
6,1998,0.737495,99.452231
7,814,0.300461,99.752692
8,346,0.127714,99.880406
9,177,0.065334,99.945740


In [91]:
plot_cdf(df, ['type_of_set_aside_code'])

type_of_set_aside_code
147731


In [31]:
column_summary = column_count_cdf(smaller_raw_df,'naics_description')

In [32]:
column_summary

,count,percent,percent_running_sum
0,181108,18.153788,18.153788
1,70069,7.023532,25.177320
2,38592,3.868360,29.045680
3,37681,3.777044,32.822724
4,36436,3.652249,36.474973
...,...,...,...
995,1,0.000100,99.999599
996,1,0.000100,99.999699
997,1,0.000100,99.999800
998,1,0.000100,99.999900
